In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# !unzip drive/My\ Drive/tiny-imagenet-200

In [1]:
import os

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras_preprocessing import image

Using TensorFlow backend.


In [0]:
train_img_gen = image.ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range = 180,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.4,
    zoom_range = 0.3,
    horizontal_flip = True
)

# we don't need to do augmentation for test data
test_img_gen = image.ImageDataGenerator(
    rescale = 1/255.0
)

In [5]:
train_gen = train_img_gen.flow_from_directory(
    'tiny-imagenet-200/train/',
    target_size = ((224, 224)),
    batch_size = 1024,
    class_mode = 'categorical'

)
test_gen = test_img_gen.flow_from_directory(
    'tiny-imagenet-200/test/',
    target_size = ((224, 224)),
    batch_size = 512,
    class_mode = 'categorical'

)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 1 classes.


In [6]:
for (x, y) in train_gen:
  print(x.shape, y.shape)
  break

(1024, 224, 224, 3) (1024, 200)


In [0]:
# validation data processing, since it is different form

class_ids = train_gen.class_indices

def load_validation_data(target_size, number_of_classes):
  with open('tiny-imagenet-200/val/val_annotations.txt') as f:
    lines = f.readlines()
    m = len(lines)
    X = np.empty((m, *target_size, 3))
    Y = np.empty(m)
    
    for i, line in enumerate(lines):
      token = line.split()
      img_name = token[0]
      img_label = token[1]
      
      img_url = os.path.join('tiny-imagenet-200/val/images', img_name)
      
      img = image.load_img(img_url)
      img = img.resize(target_size)
      X[i, ] = np.array(img, dtype=np.float32)/255.0
      img.close()
      Y[i] = class_ids[img_label]
      
    return X, keras.utils.np_utils.to_categorical(Y)
      
    

In [0]:
x_val, y_val =  load_validation_data((224, 224), 200)

In [0]:
# print(x_val.shape, y_val.shape)

In [2]:
## cifar100 data prep
from keras.datasets import cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 2s 0us/step


In [3]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [0]:
y_train = keras.utils.np_utils.to_categorical(y_train)
y_test = keras.utils.np_utils.to_categorical(y_test)

In [5]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 100)
(10000, 32, 32, 3) (10000, 100)


In [0]:
train_img_gen = image.ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range = 180,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.4,
    zoom_range = 0.3,
    horizontal_flip = True
)
test_img_gen = image.ImageDataGenerator(
    rescale = 1/255.0,
)

In [0]:
#### data augmentation
train_gen = train_img_gen.flow(
    x_train,
    y_train,
    batch_size = 512,

)
test_gen = test_img_gen.flow(
    x_test,
    y_test,
    batch_size = 512,

)

In [22]:
for (x, y) in train_gen:
  print(x.shape, y.shape)
  break

(512, 32, 32, 3) (512, 100)


In [0]:
# from google.colab import files
# files.upload()

In [0]:
#model architecture
## download resenet from here https://github.com/raghakot/keras-resnet
import resnet
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', min_delta=0.5, patience=5)


In [0]:
batch_size = 32
nb_classes = 100
nb_epoch = 100
# data_augmentation = True

# input image dimensions
img_rows, img_cols = 32, 32
# The CIFAR10 images are RGB.
img_channels = 3

In [0]:
model = resnet.ResnetBuilder.build_resnet_18((img_channels, img_rows, img_cols), nb_classes)

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 16, 16, 64)   9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 16, 16, 64)   256         conv2d_21[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 16, 16, 64)   0           batch_normalization_18[0][0]     
__________________________________________________________________________________________________
max_poolin

In [29]:
hist = model.fit_generator(train_gen, steps_per_epoch=50000//512, validation_data=test_gen, validation_steps=10000//512,  epochs=100)

Epoch 1/100
97/97 [==============================] - 34s 352ms/step - loss: 4.5979 - acc: 0.0983 - val_loss: 4.4059 - val_acc: 0.1122
Epoch 2/100
97/97 [==============================] - 29s 301ms/step - loss: 4.0700 - acc: 0.1545 - val_loss: 4.4603 - val_acc: 0.1149
Epoch 3/100
97/97 [==============================] - 30s 310ms/step - loss: 3.8699 - acc: 0.1799 - val_loss: 3.9858 - val_acc: 0.1636
Epoch 4/100
97/97 [==============================] - 30s 311ms/step - loss: 3.7289 - acc: 0.2005 - val_loss: 3.7748 - val_acc: 0.1938
Epoch 5/100
97/97 [==============================] - 30s 312ms/step - loss: 3.6144 - acc: 0.2146 - val_loss: 3.6089 - val_acc: 0.2124
Epoch 6/100
97/97 [==============================] - 30s 313ms/step - loss: 3.5085 - acc: 0.2304 - val_loss: 3.7172 - val_acc: 0.1959
Epoch 7/100
97/97 [==============================] - 30s 305ms/step - loss: 3.4420 - acc: 0.2422 - val_loss: 3.5109 - val_acc: 0.2306
Epoch 8/100
97/97 [==============================] - 30s 305ms